# ANTWERP

## Изучение источника данных

In [46]:
import pandas as pd
from sqlalchemy.sql import func

In [2]:
df=pd.read_csv("Data/listings_antwerp.csv")
df.drop("neighbourhood_group", axis=1, inplace=True)
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,50904,aplace/antwerp: cosy suite - fashion district,234077,Karin,Historisch Centrum,51.21825,4.39901,Hotel room,145,1,1,2015-05-06,0.01,4,226
1,67776,Beautiful Big House Center Antwerp,334804,Ann,Sint-Andries,51.21467,4.39225,Entire home/apt,50,7,9,2016-10-24,0.08,2,210
2,116134,Trendy Vacation Apartment Antwerp,586942,Paul,Eilandje,51.23051,4.40593,Entire home/apt,150,2,102,2020-11-16,0.93,1,351
3,224333,Large stylish room in 1930s house + garden,1167377,Geert,Deurne Zuid West,51.19772,4.45853,Private room,16,14,2,2020-07-04,0.02,2,310
4,224682,APARTMENT ROSCAM - OLD CENTRE ANTWERP,1263933,Kristien,Sint-Andries,51.21722,4.39790,Entire home/apt,70,2,344,2021-01-28,3.13,1,299


In [3]:
import warnings
warnings.filterwarnings("ignore")

! pip install sqlalchemy

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


## Создание БД

In [4]:
import sqlalchemy
from sqlalchemy import create_engine

from datetime import datetime

from sqlalchemy import (MetaData, Table, Column, Integer, Numeric, String, DateTime, 
                        Boolean, ForeignKey, create_engine, PrimaryKeyConstraint, 
                        UniqueConstraint, CheckConstraint, ForeignKeyConstraint,
                        Index, insert, BigInteger, select)

engine = create_engine('sqlite:///Antwerp.db')

In [5]:
from sqlalchemy import MetaData
metadata = MetaData()

In [6]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [7]:
listings=Table('listings', metadata,
            Column('listing_id',Integer(),primary_key=True),
            Column('name',String(50),index=True),
            Column('host_id',Integer()),
            Column('neighbourhood_id',Integer()),
            Column('latitude',Numeric(4,2)),
            Column('longitude',Numeric(4,2)),
            Column('room_type',String(50)),
            Column('price',Numeric(7,2)),
            CheckConstraint('price >= 0.00', name='listing_price_positive'),
            ForeignKeyConstraint(['host_id'], ['host.host_id']),
            ForeignKeyConstraint(['neighbourhood_id'], ['neighbourhood.neighbourhood_id']),
            extend_existing=True)

In [8]:
user=Table('user',metadata,
            Column('user_id',Integer(),primary_key=True),
            Column('user_name',String(15),nullable=False,unique=True),
            Column('email_address',String(255),nullable=False),
            Column('phone',String(20),nullable=False),
            Column('password',String(25),nullable=False),
            Column('created_on',DateTime(),default=datetime.now),
            Column('updated_on',DateTime(),default=datetime.now,onupdate=datetime.now)
           )

In [9]:
order = Table('order', metadata,
               Column('order_id', Integer(),primary_key=True),
               Column('user_id', ForeignKey('user.user_id')),
               Column('confirmed', Boolean(),default=False),
               Column('order_price', Integer()),
               extend_existing=True
              )

In [10]:
line_item = Table('line_item', metadata,
                   Column('line_item_id', Integer(), primary_key=True),
                   Column('order_id', ForeignKey('order.order_id')),
                   Column('listing_id', ForeignKey('listings.listing_id')),
                   Column('item_start_date', DateTime()),
                   Column('item_end_date', DateTime()),
                   Column('item_price', Integer()),
                   extend_existing=True
                  )

In [11]:
neighbourhood=Table('neighbourhood',metadata,
                    Column('neighbourhood_id',Integer(),primary_key=True),
                    Column('name',String(30)),
                   )

In [12]:
host=Table('host',metadata,
           Column('host_id',Integer(),primary_key=True),
           Column('name',String(30))
          )

In [13]:
room_type=Table('room_type',metadata,
                Column('room_type_id',Integer(),primary_key=True),
                Column('name',String(30))
               )

In [14]:
metadata.create_all(engine)

## Наполнение таблиц

Соединение

In [15]:
connection=engine.connect()

1. Словарь районов

In [16]:
df.head(5)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,50904,aplace/antwerp: cosy suite - fashion district,234077,Karin,Historisch Centrum,51.21825,4.39901,Hotel room,145,1,1,2015-05-06,0.01,4,226
1,67776,Beautiful Big House Center Antwerp,334804,Ann,Sint-Andries,51.21467,4.39225,Entire home/apt,50,7,9,2016-10-24,0.08,2,210
2,116134,Trendy Vacation Apartment Antwerp,586942,Paul,Eilandje,51.23051,4.40593,Entire home/apt,150,2,102,2020-11-16,0.93,1,351
3,224333,Large stylish room in 1930s house + garden,1167377,Geert,Deurne Zuid West,51.19772,4.45853,Private room,16,14,2,2020-07-04,0.02,2,310
4,224682,APARTMENT ROSCAM - OLD CENTRE ANTWERP,1263933,Kristien,Sint-Andries,51.21722,4.39790,Entire home/apt,70,2,344,2021-01-28,3.13,1,299


In [17]:
# df.info()

In [18]:
neigh_names=df["neighbourhood"].unique()
# neigh_dict={i: x for i, x in enumerate(neigh_names)}
# neigh_list=[neigh_dict]
# neigh_list
neigh_list = []
for i, x in enumerate(neigh_names):
    neigh_dict = {'neighbourhood_id': i, 'name': x}
    neigh_list.append(neigh_dict)
neigh_list[:3]

[{'neighbourhood_id': 0, 'name': 'Historisch Centrum'},
 {'neighbourhood_id': 1, 'name': 'Sint-Andries'},
 {'neighbourhood_id': 2, 'name': 'Eilandje'}]

In [19]:
neigh_dict = {x: i for i, x in enumerate(neigh_names)}
# neigh_dict

In [20]:
ins=neighbourhood.insert()
result=connection.execute(ins,neigh_list)

In [21]:
sel=select([neighbourhood])
connection.execute(sel).fetchone()

(0, 'Historisch Centrum')

In [22]:
df.head(1)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,50904,aplace/antwerp: cosy suite - fashion district,234077,Karin,Historisch Centrum,51.21825,4.39901,Hotel room,145,1,1,2015-05-06,0.01,4,226


In [23]:
listings_list = []
for rec_id in df.index:
    listings_list.append(
        {'id': df.loc[rec_id, 'id'],
         'name': df.loc[rec_id, 'name'],
         'neighbourhood_id': neigh_dict[df.loc[rec_id, 'neighbourhood']],
         'latitude': df.loc[rec_id, 'latitude'],
         'longitude': df.loc[rec_id, 'longitude'],
         'room_type': df.loc[rec_id, 'room_type'],
         'price': df.loc[rec_id, 'price'],
        }
    )
listings_list[1]

{'id': 67776,
 'name': 'Beautiful Big House Center Antwerp',
 'neighbourhood_id': 1,
 'latitude': 51.21467,
 'longitude': 4.39225,
 'room_type': 'Entire home/apt',
 'price': 50}

In [24]:
ins=listings.insert()
result=connection.execute(ins,listings_list)

In [25]:
sel=select([listings])
connection.execute(sel).fetchone()

(1, 'aplace/antwerp: cosy suite - fashion district', None, 0, Decimal('51.22'), Decimal('4.40'), 'Hotel room', Decimal('145.00'))

In [33]:
def insert_values(table, values_list):
    for value in values_list:
        ins = insert(table).values(value)
        connection.execute(ins)

In [34]:
user_list=[(1,'Nicolas','nicolas@rambler.ru','+7-929-616-88-77','@#$%890'),
           (2,'Lida','lidaok@gmail.com','+7-929-616-88-77','yyT$%333'),
           (3,'Vera','lveramuns@gmail.com','+7-353-214-12-90','yyT$%333'),
           (4,'Ivan','ivaturgenev@yandex.ru','+7-047-121-89-95','tT6^7&#20Oy'),
           (5,'Svetlana','svetaivanova@microsoft.com','+7-812-555-48-71','SD%@OUsdc7')
          ]
insert_values(user,user_list)

In [35]:
sel=select([user])
connection.execute(sel).fetchone()

(1, 'Nicolas', 'nicolas@rambler.ru', '+7-929-616-88-77', '@#$%890', datetime.datetime(2021, 4, 10, 15, 21, 42, 790318), datetime.datetime(2021, 4, 10, 15, 21, 42, 790318))

In [36]:
order_list=[(1,1,True,500),
            (2,2,True,150),
            (3,3,True,180),
            (4,1,True,200),
            (5,2,True,220)]
insert_values(order,order_list)

In [40]:
sel=select([order])
connection.execute(sel).fetchone()

(1, 1, True, 500)

In [41]:
sel=select([listings.c.price])
connection.execute(sel).fetchone()

(Decimal('145.00'),)

In [49]:
sel = select([func.max(listings.c.price)])
connection.execute(sel).scalar()

Decimal('3200.00')